In [1]:
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

In [2]:
data = ['hello world',
        'midnight',
        'calculation',
        'path',
        'short circuit']

char_set = ['<pad>'] + list(set(char for seq in data for char in seq)) 
char2idx = {char: idx for idx, char in enumerate(char_set)} 
print('char_set:', char_set)
print('char_set length:', len(char_set))

char_set: ['<pad>', 'a', 'w', 't', 'r', 'h', 'u', 'e', 'd', 'g', 'i', 'n', 's', 'm', 'p', 'l', 'c', 'o', ' ']
char_set length: 19


In [3]:
X = [torch.LongTensor([char2idx[char] for char in seq]) for seq in data]

for sequence in X:
    print(sequence)

tensor([ 5,  7, 15, 15, 17, 18,  2, 17,  4, 15,  8])
tensor([13, 10,  8, 11, 10,  9,  5,  3])
tensor([16,  1, 15, 16,  6, 15,  1,  3, 10, 17, 11])
tensor([14,  1,  3,  5])
tensor([12,  5, 17,  4,  3, 18, 16, 10,  4, 16,  6, 10,  3])


In [4]:
lengths = [len(seq) for seq in X]
print('lengths:', lengths)

lengths: [11, 8, 11, 4, 13]


In [5]:
padded_sequence = pad_sequence(X, batch_first=True) 
print(padded_sequence)
print(padded_sequence.shape)

tensor([[ 5,  7, 15, 15, 17, 18,  2, 17,  4, 15,  8,  0,  0],
        [13, 10,  8, 11, 10,  9,  5,  3,  0,  0,  0,  0,  0],
        [16,  1, 15, 16,  6, 15,  1,  3, 10, 17, 11,  0,  0],
        [14,  1,  3,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [12,  5, 17,  4,  3, 18, 16, 10,  4, 16,  6, 10,  3]])
torch.Size([5, 13])


In [6]:
sorted_idx = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True)
sorted_X = [X[idx] for idx in sorted_idx]

for sequence in sorted_X:
    print(sequence)

tensor([12,  5, 17,  4,  3, 18, 16, 10,  4, 16,  6, 10,  3])
tensor([ 5,  7, 15, 15, 17, 18,  2, 17,  4, 15,  8])
tensor([16,  1, 15, 16,  6, 15,  1,  3, 10, 17, 11])
tensor([13, 10,  8, 11, 10,  9,  5,  3])
tensor([14,  1,  3,  5])


In [7]:
packed_sequence = pack_sequence(sorted_X)
print(packed_sequence)

PackedSequence(data=tensor([12,  5, 16, 13, 14,  5,  7,  1, 10,  1, 17, 15, 15,  8,  3,  4, 15, 16,
        11,  5,  3, 17,  6, 10, 18, 18, 15,  9, 16,  2,  1,  5, 10, 17,  3,  3,
         4,  4, 10, 16, 15, 17,  6,  8, 11, 10,  3]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]), sorted_indices=None, unsorted_indices=None)


In [8]:
eye = torch.eye(len(char_set)) 
embedded_tensor = eye[padded_sequence]
print(embedded_tensor.shape)

torch.Size([5, 13, 19])


In [9]:
embedded_packed_seq = pack_sequence([eye[X[idx]] for idx in sorted_idx])
print(embedded_packed_seq.data.shape)

torch.Size([47, 19])


In [10]:
rnn = torch.nn.RNN(input_size=len(char_set), hidden_size=30, batch_first=True)


In [11]:
rnn_output, hidden = rnn(embedded_tensor)
print(rnn_output.shape)
print(hidden.shape)

torch.Size([5, 13, 30])
torch.Size([1, 5, 30])


In [12]:
rnn_output, hidden = rnn(embedded_packed_seq)
print(rnn_output.data.shape)
print(hidden.data.shape)

torch.Size([47, 30])
torch.Size([1, 5, 30])


In [13]:
unpacked_sequence, seq_lengths = pad_packed_sequence(embedded_packed_seq, batch_first=True)
print(unpacked_sequence.shape)
print(seq_lengths)

torch.Size([5, 13, 19])
tensor([13, 11, 11,  8,  4])


In [14]:
embedded_padded_sequence = eye[pad_sequence(sorted_X, batch_first=True)]
print(embedded_padded_sequence.shape)

torch.Size([5, 13, 19])


In [15]:
sorted_lengths = sorted(lengths, reverse=True)
new_packed_sequence = pack_padded_sequence(embedded_padded_sequence, sorted_lengths, batch_first=True)
print(new_packed_sequence.data.shape)
print(new_packed_sequence.batch_sizes)

torch.Size([47, 19])
tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1])
